<a href="https://colab.research.google.com/github/10Academy-Group-4/Week-4/blob/mdahwireng_preprocessing/torch_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount("/content/drive/MyDrive/")

Mounted at /content/drive/MyDrive/


In [ ]:
%cd /content/drive/MyDrive/MyDrive/10acad/week4_speech_to_text/ALFFA_PUBLIC/ASR/SWAHILI

/content/drive/MyDrive/MyDrive/10acad/week4_speech_to_text/ALFFA_PUBLIC/ASR/SWAHILI


In [ ]:
import os
os.chdir("data/train/wav")

In [ ]:
train_folders = os.listdir()

In [ ]:
waves =[]
for folder in train_folders:
  for f in os.listdir(folder):
    if f.endswith('.wav'):
      waves.append(f)

In [ ]:
os.chdir("..")

In [ ]:
os.listdir()
transcript = pd.read_csv('text', sep='\t', names=['file','transcript'])

In [ ]:
transcript.head()

,file,transcript
0,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,rais wa tanzania jakaya mrisho kikwete
1,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,yanayo andaliwa nami pendo pondo idhaa ya kisw...
2,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,inayokutangazia moja kwa moja kutoka jijini da...
3,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,juma hili bara la afrika limeshuhudia raia wa ...
4,SWH-05-20101106_16k-emission_swahili_05h30_-_0...,wakipiga kura ya maoni ilikufanya mabadiliko ya


In [ ]:
pip install torchaudio

     |████████████████████████████████| 1.9 MB 5.0 MB/s 


In [ ]:
import math, random
import torch
import torchaudio
from torchaudio import transforms
from IPython.display import Audio

class PrepSound():
  # ----------------------------
  # Load an audio file. Return the signal as a tensor and the sample rate
  # ----------------------------
  def __init__(self, audio_file, path):
    self.audio = audio_file
    self.aud_path = path + '/' + audio_file
  

  def open(self, output=False):
    samples, sr = torchaudio.load(self.aud_path)
    self.samples = samples
    self.sample_rate = sr
    if output:
      return (samples, sr)

  # ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
  def rechannel(self, new_channel, output=False):
    samples = self.samples
    sr = self.sample_rate

    if (samples.shape[0] == new_channel):
      pass

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      samples = samples[:1, :]
      self.samples = samples
    else:
      # Convert from mono to stereo by duplicating the first channel
      self.samples = torch.cat([samples, samples])
      self.samples = samples
    if output:
      return ((samples, sr))

  # ----------------------------
  # Since Resample applies to a single channel, we resample one channel at a time
  # ----------------------------
  
  def resample(self,newsr, output=False):
    samples = self.samples
    sr = self.sample_rate

    if (sr == newsr):
      pass

    num_channels = samples.shape[0]
    # Resample first channel
    resamp = torchaudio.transforms.Resample(sr, newsr)(samples[:1,:])
    if (num_channels > 1):
      # Resample the second channel and merge both channels
      resamp_dup = torchaudio.transforms.Resample(sr, newsr)(samples[1:,:])
      resamp = torch.cat([resamp, resamp_dup])
    self.samples = resamp
    self.sample_rate = newsr
    if output:
      return ((resamp, newsr))
    

  # ----------------------------
  # Pad (or truncate) the signal to a fixed length 'max_ms' in milliseconds
  # ----------------------------
  def pad_trunc(self, max_ms, output=False):
    samples = self.samples
    sr = self.sample_rate
    num_rows, samples_len = samples.shape
    max_len = sr//1000 * max_ms

    if (samples_len > max_len):
      # Truncate the signal to the given length
      samples = samples[:,:max_len]

    elif (samples_len < max_len):
      # Length of padding to add at the beginning and end of the signal
      pad_begin_len = random.randint(0, max_len - samples_len)
      pad_end_len = max_len - samples_len - pad_begin_len

      # Pad with 0s
      pad_begin = torch.zeros((num_rows, pad_begin_len))
      pad_end = torch.zeros((num_rows, pad_end_len))

      samples = torch.cat((pad_begin, sig, pad_end), 1)
      
    self.samples = samples
    self.sample_rate = sr
    if output:
      return ((samples, sr))

  # ----------------------------
  # Generate a Spectrogram
  # ----------------------------
  def spectro_gram(self, n_mels=64, n_fft=1024, hop_len=None, output=False):
    samples = self.samples
    sr = self.sample_rate
    top_db = 80

    # spec has shape [channel, n_mels, time], where channel is mono, stereo etc
    spec = transforms.MelSpectrogram(sr, n_fft=n_fft, hop_length=hop_len, n_mels=n_mels)(samples)

    # Convert to decibels
    spec = transforms.AmplitudeToDB(top_db=top_db)(spec)
    if output:
      return (spec)

In [ ]:
# ----------------------------
  # Convert the given audio to the desired number of channels
  # ----------------------------
  @staticmethod
  def rechannel(aud, new_channel):
    sig, sr = aud

    if (sig.shape[0] == new_channel):
      # Nothing to do
      return aud

    if (new_channel == 1):
      # Convert from stereo to mono by selecting only the first channel
      resig = sig[:1, :]
    else:
      # Convert from mono to stereo by duplicating the first channel
      resig = torch.cat([sig, sig])

    return ((resig, sr))

IndentationError: ignored

##**Load audio file**


##**Load transcriptions**

##**Convert into channels** 
Some of the sound files are mono (ie. 1 audio channel) while most of them are stereo (ie. 2 audio channels). Since the Neural network model expects all items to have the same dimensions, we will convert the mono files to stereo, by duplicating the first channel to the second


##**Standardize sampling rate**
We must standardize and convert all audio to the same sampling rate so that all arrays have the same dimensions.

##**Resize to the same length**
Resize to get an equal-sized audio sample by extending duration by padding it with silence, or by truncating it.

##**Data argumentation**
Perform data augmentation on the raw audio signal by applying a Time Shift to shift the audio to the left or the right by a random amount. 

##**Feature extraction**: 
Speech recognition methods derive features from the audio, such as Spectrogram or Mel Frequency Cepstrum (MFCC).

* Convert the augmented audio to a Mel Spectrogram.








**Acoustic modeling:** 

After features are extracted, these vectors are passed to acoustic models. An acoustic model attempts to map the audio signal to the basic units of speech such as phonemes or graphemes.